# API wrappers

The OpenWeatherMap offers REST endpoints for querying current weather, forecasts, historical data, etc. However, accessing this data directly via the REST API requires handling multiple API calls, query parameters, and response parsing. The pyowm library abstracts these complexities and provides useful built-in functionalities.

After signing in to OpenWeatherMap retrieve your api key at https://home.openweathermap.org/api_keys

You will also need to install the pyowm package: pip install pyowm 

In [20]:
import requests
import pyown
import json
import getpass

In [198]:
api_key = getpass.getpass("Input your token")

Input your token ········


## use case 1: managing API keys

In a raw rest API call you always have to manage credentials in each individual call. Wrappers usually store and manage the authentication for you

In [166]:
#You can get current weather data by making a GET request to an endpoint like:

params = {
    'appid' : access
}

response = requests.get('https://api.openweathermap.org/data/2.5/weather?q=London', params = params)

json.loads(response.text)

#but for every call you make using GET from now on you do need to add the parameters, since the raw API does not manage authentication for you

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 701,
   'main': 'Mist',
   'description': 'mist',
   'icon': '50d'}],
 'base': 'stations',
 'main': {'temp': 278.14,
  'feels_like': 274.93,
  'temp_min': 277.32,
  'temp_max': 279.27,
  'pressure': 1029,
  'humidity': 95,
  'sea_level': 1029,
  'grnd_level': 1024},
 'visibility': 4400,
 'wind': {'speed': 4.12, 'deg': 230},
 'clouds': {'all': 100},
 'dt': 1735486520,
 'sys': {'type': 2,
  'id': 2075535,
  'country': 'GB',
  'sunrise': 1735459572,
  'sunset': 1735487934},
 'timezone': 0,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

Most wrappers (pyowm included) include some way of initializing a session with the authentication key that you then don't need to type again.

Initialize pyowm with the default configuration. Thenopen the weather manager

Check out a snippet here: https://pyowm.readthedocs.io/en/latest/v3/code-recipes.html#weather_data

In [30]:
!pip install pyowm

In [34]:
#using an open weather map API wrapper 
from pyowm.owm import OWM
owm = OWM(api_key)
weather_mgr = owm.weather_manager()

## use case 2: Simplified calls

With the raw REST API, you'd have to build a URL manually, send the request, and parse the JSON response to get the current weather.

In [90]:
#Using an open weather map RAW REST API
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'
params = {
    'appid': api_key,
    'units': 'metric' } # Use 'metric' to get temperature in Celsius
response = requests.get(url,params= params)
data = response.json()
temperature = data['main']['temp']
humidity = data['main']['humidity']
wind_speed = data['wind']['speed']

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 4.39°C, Humidity: 94%, Wind Speed: 4.12 m/s


Get the equivalent call as above for the city of London using the pyowm package

In [94]:
#using an open weather map API wrapper 
from pyowm.owm import OWM
owm = OWM(api_key)
weather_mgr = owm.weather_manager()
London = weather_mgr.weather_at_place("London,GB")
London_weather = London.weather
temp = London_weather.temperature("celsius")
Temperature_02 =temp["temp"]
wind = London_weather.wind()
wind_speed_02 = wind["speed"]
humid_02 = London_weather.humidity
print(f"Temperature: {Temperature_02}°C, Humidity: {humid_02}%, Wind Speed: {wind_speed_02} m/s")

Temperature: 4.5°C, Humidity: 94%, Wind Speed: 4.12 m/s


## use case 3: Combining and chaining calls

Wrappers often offer methods that make multiple calls to batch requests that make sense to batch. And often they offer methods that make sequences of calls that each returns information necessary to make the next call.

For example, to get a weather forecast for a specific city using the raw API you need to first geocode the city to get its latitude and longitude:

In [168]:
city = 'New York'
geocode_url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'
geocode_response = requests.get(geocode_url,params=params).json()

lat = geocode_response['coord']['lat']
lon = geocode_response['coord']['lon']

Then, request the weather forecast for that latitude/longitude:

In [220]:
forecast_url = f'http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}'
params = {
    'appid': api_key,
    'units': 'metric' } # Use 'metric' to get temperature in Celsius
forecast_response = requests.get(forecast_url, params=params).json()

for entry in forecast_response['list']:
    print(f"Time: {entry['dt_txt']}, Temp: {entry['main']['temp']}°C")

Time: 2024-12-29 18:00:00, Temp: 9.46°C
Time: 2024-12-29 21:00:00, Temp: 9.69°C
Time: 2024-12-30 00:00:00, Temp: 9.69°C
Time: 2024-12-30 03:00:00, Temp: 10.43°C
Time: 2024-12-30 06:00:00, Temp: 10.37°C
Time: 2024-12-30 09:00:00, Temp: 10.68°C
Time: 2024-12-30 12:00:00, Temp: 10.54°C
Time: 2024-12-30 15:00:00, Temp: 10.95°C
Time: 2024-12-30 18:00:00, Temp: 10.49°C
Time: 2024-12-30 21:00:00, Temp: 10.09°C
Time: 2024-12-31 00:00:00, Temp: 8.8°C
Time: 2024-12-31 03:00:00, Temp: 7.75°C
Time: 2024-12-31 06:00:00, Temp: 6.66°C
Time: 2024-12-31 09:00:00, Temp: 5.36°C
Time: 2024-12-31 12:00:00, Temp: 4.63°C
Time: 2024-12-31 15:00:00, Temp: 6.34°C
Time: 2024-12-31 18:00:00, Temp: 8.5°C
Time: 2024-12-31 21:00:00, Temp: 9.12°C
Time: 2025-01-01 00:00:00, Temp: 9.54°C
Time: 2025-01-01 03:00:00, Temp: 9.36°C
Time: 2025-01-01 06:00:00, Temp: 8.17°C
Time: 2025-01-01 09:00:00, Temp: 7.12°C
Time: 2025-01-01 12:00:00, Temp: 6.35°C
Time: 2025-01-01 15:00:00, Temp: 7.71°C
Time: 2025-01-01 18:00:00, Temp: 5.

Two calls: one for geocoding, one for forecasts.
But with pyowm, because this is a common operation, there is a method that handles the geocoding internally and then fetches the weather forecast in one step.

Get the above forecast in a single call using pyowm.

Hint: search for "forecast_at_place" in the code recipies of the documentation

In [226]:
#using an open weather map API wrapper to get the weather forcast for Newyork
from pyowm.owm import OWM
owm = OWM(api_key)
weather_mgr = owm.weather_manager()
daily_forcast = weather_mgr.forecast_at_place('New York,US', '3h')

In [228]:
# Access the weather objects in the forecast
for weather in daily_forcast.forecast.weathers:
    temp = weather.temperature('celsius')  # Get temperature in Celsius
    print(f"Time: {weather.reference_time('iso')}, Temp: {temp['temp']}°C")

Time: 2024-12-29 18:00:00+00:00, Temp: 9.55°C
Time: 2024-12-29 21:00:00+00:00, Temp: 9.73°C
Time: 2024-12-30 00:00:00+00:00, Temp: 9.69°C
Time: 2024-12-30 03:00:00+00:00, Temp: 10.43°C
Time: 2024-12-30 06:00:00+00:00, Temp: 10.37°C
Time: 2024-12-30 09:00:00+00:00, Temp: 10.68°C
Time: 2024-12-30 12:00:00+00:00, Temp: 10.54°C
Time: 2024-12-30 15:00:00+00:00, Temp: 10.95°C
Time: 2024-12-30 18:00:00+00:00, Temp: 10.49°C
Time: 2024-12-30 21:00:00+00:00, Temp: 10.09°C
Time: 2024-12-31 00:00:00+00:00, Temp: 8.8°C
Time: 2024-12-31 03:00:00+00:00, Temp: 7.75°C
Time: 2024-12-31 06:00:00+00:00, Temp: 6.66°C
Time: 2024-12-31 09:00:00+00:00, Temp: 5.36°C
Time: 2024-12-31 12:00:00+00:00, Temp: 4.63°C
Time: 2024-12-31 15:00:00+00:00, Temp: 6.34°C
Time: 2024-12-31 18:00:00+00:00, Temp: 8.5°C
Time: 2024-12-31 21:00:00+00:00, Temp: 9.12°C
Time: 2025-01-01 00:00:00+00:00, Temp: 9.54°C
Time: 2025-01-01 03:00:00+00:00, Temp: 9.36°C
Time: 2025-01-01 06:00:00+00:00, Temp: 8.17°C
Time: 2025-01-01 09:00:00+00:

## use case 4: Convenience methods

Wrappers often offer built-in methods to handle common kinds of tasks related to the APIs, reducing the need for manual calculations.

for example converting units (e.g., temperature from Celsius to Fahrenheit) or working with more complex data requires manual conversion when using the raw API.

In [232]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'
params = {
    'units': 'metric' } # Use 'metric' to get temperature in Celsius
response = requests.get(url, params = params)
data = response.json()
temperature_celsius = data['main']['temp']
temperature_fahrenheit = (temperature_celsius * 9/5) + 32

print(f"Temperature in Celsius: {temperature_celsius}°C, Fahrenheit: {temperature_fahrenheit}°F")

Temperature in Celsius: 5.21°C, Fahrenheit: 41.378°F


But the pyowm wrapper offers built-in methods to handle these kinds of tasks, reducing the need for manual calculations.
Get the temperature both in Celcius and Farenheit using pyowm. Navigate the code recipes to figure out the inbuilt methods for this.

In [246]:
#using an open weather map API wrapper to get London temperature in both fahrenheit and celcius
from pyowm.owm import OWM
owm = OWM(api_key)
weather_mgr = owm.weather_manager()

#get weather details for London
London = weather_mgr.weather_at_place("London,GB")
London_weather = London.weather

#get temperature in celcius and Fahrenheit
temp_c = London_weather.temperature("celsius")["temp"]
temp_F = London_weather.temperature("fahrenheit")["temp"]

In [250]:
temp_C = temp["temp"]
temp_F = temp_F["temp"]

TypeError: 'float' object is not subscriptable

In [238]:
print(f"Temperature in Celsius: {temp}°C, Fahrenheit: {temp_F}°F")

Temperature in Celsius: {'temp': 5.21, 'temp_max': 6.54, 'temp_min': 4.45, 'feels_like': 2.36, 'temp_kf': None}°C, Fahrenheit:<bound method Weather.temperature of <pyowm.weatherapi25.weather.Weather - reference_time=2024-12-29 16:25:06+00:00, status=clouds, detailed_status=overcast clouds>>°F


In [240]:
print(f"Temperature in Celsius: {temp}°C, Fahrenheit: {temp_F}°F")

Temperature in Celsius: {'temp': 5.21, 'temp_max': 6.54, 'temp_min': 4.45, 'feels_like': 2.36, 'temp_kf': None}°C, Fahrenheit: <bound method Weather.temperature of <pyowm.weatherapi25.weather.Weather - reference_time=2024-12-29 16:25:06+00:00, status=clouds, detailed_status=overcast clouds>>°F


In [242]:
temp

{'temp': 5.21,
 'temp_max': 6.54,
 'temp_min': 4.45,
 'feels_like': 2.36,
 'temp_kf': None}